## Ejercicios Airtable

In [ ]:
import numpy as np
import pandas as pd
from time import sleep

import requests

from pprint import pprint

### Ejercicio 1:
- Crear una tabla en **Airtable** y cargar los datos de las recetas de ensaladas del ejercicios de **Web Scraping** con **BeautifulSoup**.

- Si tienes algúna otra información o datos puedes usarlos para hacer este ejercicio.

- **Recuerda que el nombre y el tipo de dato de las columnas del DataFrame deben ser iguales a las columnas de Airtable.**

In [ ]:
API_KEY = "keyvnQcc1FFCaFsDR" # Usuario

BASE_ID = "appMXgIxF5B3XuhQL" # Base: Tabla API

TABLE_ID = "tblWOKyz5kU18ueit" # Tabla: Ensaladas API

airtable_base_url = "https://api.airtable.com/v0"

# Headers
headers = {"Authorization" : f"Bearer {API_KEY}",
           "Content-Type"  : "application/json"}

In [ ]:
# Leemos el DataFrame y nos seguramos que el tipo de dato y el nombre de cada columna sea igual al de las columnas en Airtable
df = pd.read_csv("todas_recetas.csv")

df.info()

In [ ]:
# Llenamos todos los NaN's

df.fillna("", inplace = True)

In [ ]:
df.info()

In [ ]:
df = df[df.columns].astype(str)

In [ ]:
# Create Records

# Endpoint
endpoint = f"{airtable_base_url}/{BASE_ID}/{TABLE_ID}"

print(endpoint)

In [ ]:
datos_json = [{"fields" : df.iloc[i, :].to_dict()} for i in range(df.shape[0])]

pprint(datos_json)

In [ ]:
for i in range(0, df.shape[0], 10):
    
    data = {"records" : datos_json[i : i + 10]}
    
    response = requests.post(url = endpoint, json = data, headers = headers) # POST
    
    sleep(0.5)

### Ejercicio 2:
- Extrae todos los datos de la tabla de **Airtable** y transformalo en un DataFrame.
- Elimina la tabla usando el endpoint **DELETE**.

In [ ]:
# GET Records

params = {"offset" : None}

df_airtable = pd.DataFrame()

while params.get("offset") != None or df_airtable.shape[0] == 0:
    
    response = requests.get(url = endpoint, headers = headers, params = params)
    
    print(response.url)
    
    print(f"response: {response.status_code}")
    
    params["offset"] = response.json().get("offset")
    
    print(params.get("offset"))
    
    df_airtable = pd.concat([df_airtable, pd.json_normalize(response.json()["records"])], ignore_index = True)
    
    sleep(0.5)
    
df_airtable.shape[0]

In [ ]:
df_airtable.columns = [x.split(".")[1] if "." in x else x for x in df_airtable.columns]

df_airtable

In [ ]:
# DELETE Records

for i in range(0, df_airtable.shape[0], 10):

    params = "&".join([f"records[]={id_}" for id_ in df_airtable["id"][i : i + 10]])
    
    response = requests.delete(url = endpoint, params = params, headers = headers) # DELETE

    print(f"response: {response.status_code}")

    print(f"endpoint: {response.url}")

#     print("-"*120)

#     pprint(response.json(), sort_dicts = False)

    print("-"*120)
    
    sleep(0.5)

In [ ]:
################################################################################################################################